In [20]:
for i in range(10):
    print(i)

# next(range(0, 10, 1))

0
1
2
3
4
5
6
7
8
9


In [ ]:
# yield

In [22]:
a = [_ for _ in range(10)]

for i in a:
    print(i)

# next(a.__iter__)

0
1
2
3
4
5
6
7
8
9


Итераторы и Python
---

*Итератор* - паттерн проектирования, который предоставляет удобный интерфейс для перебора какой-либо коллекции элементов.

В качестве примера приведём пользовательский класс "RecordCollection", хранящий объекты класса "Record" в хэш-таблице, где в качестве хэша выступают тэги, назначаемые пользователем.

In [1]:
class Record:
    def __init__(self, *tags, **data):
        self.__dict__["_tags"] = list(tags)
        self.__dict__["_data"] = data
    
    def __getitem__(self, key):
        if (key in self._data.keys()):
            return self._data[key]
        raise KeyError
    
    def __setattr__(self, __name: str, __value):
        self.__dict__["_data"][__name] = __value
    
    def __getattr__(self, __name: str):
        if __name in self.__dict__["_data"].keys():
            return self.__dict__["_data"][__name]
        raise AttributeError

    def __repr__(self) -> str:
        return f"tags:\t{self._tags}\ndata:\t{self._data}"
    
    def __str__(self) -> str:
        return self.__repr__()
    
    def addTag(self, tag: str):
        self.__dict__["_tags"].append(tag)

    def getTags(self):
        return self.__dict__["_tags"]

In [28]:
rec = Record()
rec.value = 30
rec.addTag("t1")
rec

tags:	['t1']
data:	{'value': 30}

In [2]:
a = Record("sample_tag", first_val = 1, second_val = 2)
print(a)
print(a.first_val)
b = Record()
b.addTag("sample_tag")
print(b)

tags:	['sample_tag']
data:	{'first_val': 1, 'second_val': 2}
1
tags:	['sample_tag']
data:	{}


Ключевые особенности итераторов:
- Алгоритм перебора коллекции определён в итераторе, исходная коллекция знает только о существовании итератора
- Итерация реализуется через "магический метод" \_\_iter__ внутри коллекции, метод возвращает ссылку на экземпляр итератора
- Протокол итератора состоит в двух методах: 
    - \_\_next__ - возвращает ссылку на следующий элемент коллекции
    - \_\_iter__ - возвращает ссылку на сам объект итератора (договорённость)

In [15]:
class RecordIterator:
    def __init__(self, records:dict):
        self.__records = records
        self.__tags = list(self.__records.keys())
        self.__current_tag = 0 if len(self.__tags) > 0 else None
        self.__current_index = 0 if len(self.__tags) > 0 else None
        self.__iterated = set()
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.__current_tag is None or self.__current_index is None:
            raise StopIteration
        
        out = self.__records[self.__tags[self.__current_tag]][self.__current_index]

        while(out in self.__iterated):
            self.__iterate()
            if self.__current_tag is None or self.__current_index is None:
                raise StopIteration
            out = self.__records[self.__tags[self.__current_tag]][self.__current_index]
        
        self.__iterated.add(out)

        return out

    def __iterate(self):
        self.__current_index = self.__current_index + 1 
        # if we overstepped list bound
        if self.__current_index >= len(self.__records[self.__tags[self.__current_tag]]):
            self.__current_index = None
            # try to get next tag
            self.__current_tag = self.__current_tag + 1 
            # if reached end of tag list set both current index and tag None
            if self.__current_tag >= len(self.__tags):
                self.__current_tag = None
            else:
                # else set current index to first element of list for this tag
                self.__current_index = 0
    
    def begin(self):
        if len(self.__tags) > 0:
            return self.__records[self.__tags[self.__current_index]]
        return None
    
    def end(self):
        if len(self.__tags) > 0:
            self.__current_index = len(self.__records) - 1
            return self.__records[self.__tags[self.__current_index]]

Положим, нам необходимо пройтись по экземпляру класса "RecordCollection" в цикле for так, чтобы каждая запись была выведена ровно один раз - если у неё несколько тэгов, то по первому из них

In [16]:
class RecordCollection:
    def __init__(self, *records):
        self._recs = {}
        for rec in records:
            if not type(rec) is Record:
                raise ValueError
            for tag in rec.getTags():
                if tag in self._recs.keys():
                    self._recs[tag].append(rec)
                else:
                    self._recs[tag] = [rec]
    
    def __iter__(self):
        return RecordIterator(self._recs)
    
    def addRecord(self, rec:Record):
        if not type(rec) is Record:
            raise ValueError
        self._recs.append(rec)


In [31]:
a = Record("t1", "t2", a = 1)
b = Record("t2", b = "Lorem Ipsum")
c = Record("t1", "t2", c = (1, 2, 3))


In [34]:

collection = RecordCollection(a, b, c)
collection

In [35]:
for rec in collection:
    print(rec)

tags:	['t1', 't2']
data:	{'a': 1}
tags:	['t1', 't2']
data:	{'c': (1, 2, 3)}
tags:	['t2']
data:	{'b': 'Lorem Ipsum'}


In [36]:
for rec in RecordCollection():
    print("Lorem ipsum")